In [1]:
from nltk.corpus import wordnet
import nltk
import MeCab
import unicodedata
import pandas as pd

# traindf=pd.read_csv("data/train.csv")
# testdf=pd.read_csv("data/test.csv")

import mojimoji
import re
import jaconv


def parse_text(text, debug=False):
    '''
    Get location
    '''

    text = mojimoji.zen_to_han(text, kana=False)
    text = re.sub(r'[\(（[].*[）\)]]', '', text)
#     text = re.sub(r'[\s、]', '', text)
#     text = re.sub(r'―', '', text)
#     text = re.sub(r'…', '', text)
#     text = re.sub(r'[「.*」]', '', text)
    text = re.sub(r'/＼', '', text)
    text = re.sub(r'[0-9]', '0', text)
    text = re.sub(r'[[#\.*]\]', '', text)

    text = jaconv.kata2hira(text)
    return text

# df=pd.concat([traindf,testdf])
# df.body=df.body.map(parse_text)
# traindf.body=traindf.body.map(parse_text)
# testdf.body=testdf.body.map(parse_text)


def normalize(text):
    normalized_text = normalize_unicode(text)
    normalized_text = normalize_number(normalized_text)
    normalized_text = lower_text(normalized_text)
    return normalized_text


def lower_text(text):
    return text.lower()


def normalize_unicode(text, form='NFKC'):
    normalized_text = unicodedata.normalize(form, text)
    return normalized_text


def lemmatize_term(term, pos=None):
    if pos is None:
        synsets = wordnet.synsets(term)
        if not synsets:
            return term
        pos = synsets[0].pos()
        if pos == wordnet.ADJ_SAT:
            pos = wordnet.ADJ
    return nltk.WordNetLemmatizer().lemmatize(term, pos=pos)


def normalize_number(text):
    """
    pattern = r'\d+'
    replacer = re.compile(pattern)
    result = replacer.sub('0', text)
    """
    # 連続した数字を0で置換
    replaced_text = re.sub(r'\d+', '0', text)
    return replaced_text


# mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
# mecab=MeCab.Tagger ('-d /usr/local/lib/mecab/dic/UniDic-kindai_1603')
mecab = MeCab.Tagger('-Owakati')

# 形態素解析をして、名詞だけ取り出す


def tokenize(text):
    available_norm = ['接尾', '一般', '形容動詞語幹', 'サ変接続']
    node = mecab.parseToNode(text)
    l = []
    while node:

        l.append(node.surface)
        node = node.next
    return ' '.join(l)


# 記事群のdictについて、形態素解析をしてリストに返す
def get_words(contents):
    available_norm = ['接尾', '一般', '形容動詞語幹', 'サ変接続']
    node = mecab.parseToNode(contents)
    l = []
    while node:

        l.append(node.surface)
        node = node.next
    return l

# 一つの記事を形態素解析して返す


def get_words_main(content):
    return [token for token in tokenize1(content)]


def get_words_main1(content):
    retoken = ""
    for token in tokenize(content):
        retoken += token+" "
    return retoken

In [2]:
from sklearn.feature_extraction.text import CountVectorizer


def preprocess():
    traindf = pd.read_csv("data/train.csv")
    testdf = pd.read_csv("data/test.csv")

    df = pd.concat([traindf, testdf])
    df.body = df.body.map(parse_text).map(normalize).map(tokenize)
    count = CountVectorizer()
    bags = count.fit_transform(df.body)
    # print(bags.toarray())

    features = count.get_feature_names()
    # # print(features)

    bodyvec = pd.DataFrame(bags.toarray(), columns=features)
    newdf = pd.concat([df.reset_index(drop=True), pd.DataFrame(bodyvec)], axis=1)
    train = newdf.dropna().drop(["writing_id", "body", ], axis=1)
    test = newdf[newdf.author.isnull()].drop(["writing_id", "body"], axis=1)
    test = test.drop(["author"], axis=1)
    X = train.drop(["author"], axis=1)
    y = train.author
    return X, y, test


X, y, test = preprocess()

/Users/ogatatakuya/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/Users/ogatatakuya/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: Possible nested set at position 1


In [ ]:
X

In [95]:
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.2, random_state=0)  # 80%のデータを学習データに、20%を検証データにする

In [99]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()  # ロジスティック回帰モデルのインスタンスを作成
lr.fit(train_X, train_y)  # ロジスティック回帰モデルの重みを学習
pred = lr.predict(test_X)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[607   1]
 [  5  50]]
accuracy =  0.9909502262443439
precision =  0.9803921568627451
recall =  0.9090909090909091
f1 score =  0.9433962264150944


In [97]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

mlp = MLPClassifier(
    **{"hidden_layer_sizes": (128, 128, 128, 128, 128, 128), "random_state": 42})
mlp.fit(train_X, train_y)
pred = mlp.predict(test_X)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[608   0]
 [  3  52]]
accuracy =  0.995475113122172
precision =  1.0
recall =  0.9454545454545454
f1 score =  0.9719626168224299


In [98]:
mlp = MLPClassifier(
    **{"hidden_layer_sizes": (128, 128, 128, 128, 128, 128), "random_state": 42})
mlp.fit(X, y)
pred = mlp.predict(test)
# pred = model.predict(np.array(test))
pred = np.where(pred > 0.5, 1, 0)
sub = pd.DataFrame(pd.read_csv("data/test.csv")['writing_id'])
sub["author"] = list(pred)
sub.to_csv("submission.csv", index=False)

In [48]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
x_train = tokenizer.texts_to_sequences(X)
x_test = tokenizer.texts_to_sequences(test)


In [37]:
for text, vector in zip(X.head(3), x_train[0:3]):
    print(text)
    print(vector)

 先ごろ の 本欄 に 僕 の 風 報 に かい た 天皇陛下 に 捧ぐる 言葉 を 評し て 俗 うけ を 狙っ た 媚態 露出 だ と の こと で ある が 白井 明 先生 の 鑑賞 眼 は 浅薄 低俗 と 申さ なけれ ば なら ない 。 あの 文章 に こもる 祖国 へ よせる 僕 の 愛情 や あれ を 書か ず に い られ なかっ た 情熱 を 読みとる こと が でき ない と は 白井 先生 が 頃日 書く 意味 も ない 駄文 ばかり 書い てる せい な の で ある 。 いったい に 文学 の 反語 性 に 味読 の 及ば ぬ 識見 低俗 な やから が 文学 を 批評 する という の が 間違っ て いる 。 僕 の 堕落 論 その他 の えっ せい に し て も 小説 に し て も その 反語 に こもる 正しい 意味 を 理解 し 得 ず に 軽率 な 判読 断定 を 下す から 読者 に 誤読 の お手本 を 与え て いる よう な もの で ある 。 本名 で は 愚かしい そら ごと しか 書け ず 匿名 で しか 本音 の 吐け ぬ 文学 者 など という もの は ない 。 僕 に は 匿名 の 必要 は ない 。 いつ でも 本音 を 吐き ぎりぎり の こと を 言っ てる から だ 。 だから また ぼく の 本音 は 文学 の 本質 的 な もの で あり 単なる 中傷 の けち くさい 汚らし さ は ない の で ある 。 白井 明 先生 も 本名 で 本音 を 吐く こと を 学び た ま え 。 本名 で 君 の けち あさまし さ を さらけだす こと の 苦痛 に 堪え て その 争い の 嵐 の 中 で 自分 を 育て た ま え 。 さ すれ ば 文学 の 本質 に も やがて 近づき うる で あろ う 。 天皇陛下 に さ ゝ ぐる 言葉 に こもる 大いなる 愛情 も やみ がたい 情熱 も 君 の 目 に は 逆 の 意味 に うつる の も きわめて 当然 な こと で ある 。 しかし こういう 愚 に も つか ない 批評 で も それ が 君 の 本音 なら 仕方 が ない から せめて 本名 で 書か れん こと を 。 さ すれ ば 進歩 は あり うる で あろ う 。 

In [49]:
max_len=2000
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

In [50]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

vocabulary_size = len(tokenizer.word_index) + 1  # 学習データの語彙数+1

model = Sequential()

model.add(Embedding(input_dim=vocabulary_size, output_dim=32))
model.add(LSTM(16, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          3446624   
_________________________________________________________________
lstm_4 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 3,449,777
Trainable params: 3,449,777
Non-trainable params: 0
_________________________________________________________________


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import warnings
# display(HTML("<style>.container { width:80% !important; }</style>"))
warnings.filterwarnings('ignore')
train_X, valid_X, train_y, valid_y = train_test_split(x_train, y, test_size = 0.2, random_state = 0) # 80%のデータを学習データに、20%を検証データにする


In [52]:
history = model.fit(
    train_X, train_y, batch_size=32, epochs=10,
    validation_data=(valid_X, valid_y)
)

Train on 2649 samples, validate on 663 samples
Epoch 1/10
2649/2649 [==============================] - 95s 36ms/step - loss: 0.3738 - accuracy: 0.9154 - val_loss: 0.2815 - val_accuracy: 0.9170
Epoch 2/10
2649/2649 [==============================] - 94s 36ms/step - loss: 0.2516 - accuracy: 0.9256 - val_loss: 0.2605 - val_accuracy: 0.9170
Epoch 3/10
2649/2649 [==============================] - 100s 38ms/step - loss: 0.1652 - accuracy: 0.9339 - val_loss: 0.1363 - val_accuracy: 0.9336
Epoch 4/10
2649/2649 [==============================] - 123s 47ms/step - loss: 0.0601 - accuracy: 0.9879 - val_loss: 0.1597 - val_accuracy: 0.9427
Epoch 5/10
2649/2649 [==============================] - 120s 45ms/step - loss: 0.0308 - accuracy: 0.9977 - val_loss: 0.2075 - val_accuracy: 0.9502
Epoch 6/10
2649/2649 [==============================] - 120s 45ms/step - loss: 0.0200 - accuracy: 0.9985 - val_loss: 0.1294 - val_accuracy: 0.9563
Epoch 7/10
2649/2649 [==============================] - 108s 41ms/step - 

In [54]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
pred = model.predict_classes(valid_X)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[580  28]
 [  6  49]]
accuracy =  0.9487179487179487
precision =  0.6363636363636364
recall =  0.8909090909090909
f1 score =  0.7424242424242423
